In [1]:
 # Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

In [2]:
# Load in Weather Data
file_path = "output_data\cities.csv"

cities_df = pd.read_csv(file_path, encoding = 'utf-8')
cities_df

,City ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,1,port-gentil,40.0,GA,1595894770,94.0,-0.72,8.78,75.200,6.09
1,2,bulgan,13.0,MN,1595894771,74.0,48.81,103.53,60.530,0.42
2,3,matamoros,90.0,MX,1595894771,88.0,25.53,-103.25,73.400,4.06
3,4,nikolskoye,19.0,RU,1595894657,82.0,59.70,30.79,60.008,3.00
4,5,tuatapere,100.0,NZ,1595894691,87.0,-46.13,167.68,42.998,0.89
...,...,...,...,...,...,...,...,...,...,...
545,546,lasa,5.0,IT,1595894822,93.0,46.62,10.70,60.800,1.50
546,547,buala,69.0,SB,1595894760,74.0,-8.14,159.59,84.236,3.91
547,548,saint anthony,1.0,US,1595894822,42.0,45.02,-93.22,82.400,5.70
548,549,lingyuan,20.0,CN,1595894822,74.0,41.24,119.40,72.968,3.07


In [3]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)

# Store 'Lat' and 'Lng' into Locations 
locations = cities_df[["Lat", "Lng"]].astype(float)

# Store Humidity for weights
humidity = cities_df["Humidity"].astype(float)


In [4]:
# Create a Heatmap layer
gmaps.configure(api_key=g_key)
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 3)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [5]:
# Configure Dataset to Fit Ideal Vacation
hotel_df = cities_df[cities_df['Max Temp'] >= 65]
hotel_df = hotel_df[hotel_df['Max Temp'] <= 75]
hotel_df = hotel_df[hotel_df['Humidity'] <= 50]
hotel_df = hotel_df[hotel_df['Cloudiness'] <= 50]
hotel_df = hotel_df[hotel_df['Wind Speed'] <= 7]
hotel_df = hotel_df[hotel_df['Country'] != 'US']
hotel_df = hotel_df[hotel_df['Lat'] <= 70]
hotel_df = hotel_df[hotel_df['Lat'] >= -70]
hotel_df = hotel_df
hotel_df.reindex()

,City ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
101,102,jatai,0.0,BR,1595894780,41.0,-17.88,-51.71,68.090,1.96
197,198,jardim,0.0,BR,1595894789,45.0,-21.48,-56.14,71.888,1.21
286,287,boysun,0.0,UZ,1595894798,30.0,38.21,67.20,69.962,0.98
456,457,aksu,6.0,CN,1595894813,41.0,41.12,80.26,72.428,0.38
514,515,passagem franca,50.0,BR,1595894819,40.0,-6.18,-43.78,74.930,1.74


In [6]:
# Hotel Data Retrieval
# Param constants, 10000m was used since a few of the locations did not have a hotel within 5000m
target_type = "lodging"
radius = 10000

# Storage Dictionary
hotels = {}

# Hotel Columns
hotel_df['Hotel Name'] = ''
hotel_df['Hotel Rating'] = ''
hotel_df['Hotel Reviews'] = ''
hotel_df['Hotel Address'] = ''

# Row Counter
counter = 0

# Places API
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# Data Retrieval
for row in hotel_df.iterrows():
    params = {
        "location": f"{row[1]['Lat']}, {row[1]['Lng']}",
        "types": target_type,
        "radius": radius,
        "key": g_key
    }
    hotels[row[1]['City']] = requests.get(base_url, params).json()['results']
    row[1]['Hotel Name'] = hotels[row[1]['City']][0]['name']
    try:
        row[1]['Hotel Rating'] = hotels[row[1]['City']][0]['rating']
    except:
        row[1]['Hotel Rating'] = None
    try:
        row[1]['Hotel Reviews'] = hotels[row[1]['City']][0]['user_ratings_total']
    except:
        row[1]['Hotel Reviews'] = None
    try:
        row[1]['Hotel Address'] = hotels[row[1]['City']][0]['vicinity']
    except:
        row[1]['Hotel Address'] = None
    hotel_df.iloc[counter,:] = row[1]
    counter += 1
    if row[1]['City'] == list(hotel_df['City'])[0]:
        print(f"Within 10,000m there are at least...")
    print(f"        {len(hotels[row[1]['City']])} lodging establishments from {row[1]['City'].capitalize()}, {row[1]['Country']}")


Within 10,000m there are at least...
        20 lodging establishments from Jatai, BR
        19 lodging establishments from Jardim, BR
        4 lodging establishments from Boysun, UZ
        20 lodging establishments from Aksu, CN
        6 lodging establishments from Passagem franca, BR


In [7]:
# Print Final Hotel Dataframe
hotel_df

,City ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name,Hotel Rating,Hotel Reviews,Hotel Address
101,102,jatai,0.0,BR,1595894780,41.0,-17.88,-51.71,68.090,1.96,Hotel Califórnia,3.8,256,"Avenida Miguel Gonçalves da Silva, 754 - Vila ..."
197,198,jardim,0.0,BR,1595894789,45.0,-21.48,-56.14,71.888,1.21,Hotel Estância,4.5,273,"Rua Duque de Caxias, 2041 - Centro, Guia Lopes..."
286,287,boysun,0.0,UZ,1595894798,30.0,38.21,67.20,69.962,0.98,Gaza Hotel,4.8,16,Baysun
456,457,aksu,6.0,CN,1595894813,41.0,41.12,80.26,72.428,0.38,Pudong Holiday Hotel,None,None,"交通西路1号, Jiaotong Road, Aksu City, Aksu Prefecture"
514,515,passagem franca,50.0,BR,1595894819,40.0,-6.18,-43.78,74.930,1.74,Hotel Brasil,None,None,Passagem Franca


In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]
print(locations)

       Lat    Lng
101 -17.88 -51.71
197 -21.48 -56.14
286  38.21  67.20
456  41.12  80.26
514  -6.18 -43.78


In [9]:
# Add marker layer ontop of heat map
hotel_layer = gmaps.marker_layer(
    locations,
    info_box_content=hotel_info,
    display_info_box=True
)


fig.add_layer(hotel_layer)
fig

Figure(layout=FigureLayout(height='420px'))